In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from scipy.stats import norm
from copy import deepcopy
from scipy.stats import t as t_dist
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.api as sm

## 1 Cost calculation

### 1.1 Change with prior mean - Uniform-Uniform setting 

In [ ]:
K = 100 # number of experiments
N = 10  # number of samples in each experiment
std = 3 # standard deviation of noise
f_x = 4 # number of historical features
Z = t_dist.ppf(0.975,N - f_x - 2) # critical value for two-tailed test at 95% confidence level

cost = np.zeros((5,5,1000))

for index1,anchor_mean in enumerate([1,2,3,4,5]):
    for time_seq in tqdm(range(1000)):

        # Generate true parameters
        true_tao = np.random.uniform(anchor_mean - np.sqrt(9*12)/2, anchor_mean + np.sqrt(9*12)/2, K)
        optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)]) 
        ture_theta_vector = np.random.uniform(-0.3,0.5,(K,f_x + 1))

        # Generate historical features and response variable
        X = np.ones((K,f_x+2,N))
        for i in range(K):
            for j in range(N):
                indicator = np.random.choice([0,1],1)
                if indicator == 0:
                    X[i,-1,j] = 0
                else:
                    X[i,-1,j] = 1
            hist_feature = np.random.uniform(0,1,(f_x,N))
            X[i,1:-1,:] = hist_feature
        Y = np.zeros((K,N))
        for i in range(K):
            for j in range(N):
                Y[i,j] = np.sum(ture_theta_vector[i] * X[i,0:-1,j]) + true_tao[i]*X[i,-1,j] + np.random.uniform(-np.sqrt(std**2*12)/2,np.sqrt(std**2*12)/2)

        #ols
        tao_hat = np.zeros(K)
        variance = np.zeros(K)
        p_value_list = np.zeros(K)
        b_list = np.zeros(K)

        variance_1 = np.zeros(K)

        for k in range(K):
            model = sm.OLS(Y[k], X[k].T).fit()
            tao_hat[k] = model.params[-1]
            p_value_list[k] = model.pvalues[-1]
            b_list[k] = N*np.linalg.inv(X[k] @ X[k].T + (10**-5) * np.eye(X[k].shape[0]))[-1,-1]
            variance[k] = np.sum(model.resid**2)/(N - f_x - 2)

         

        #pesonlised parameter and consistant parameter
        tao_0 = np.mean(tao_hat)
        beta_vector = np.zeros(K)

        estimated_noise_variance = np.mean(variance)
        denominator = np.mean((tao_hat - tao_0)**2) - estimated_noise_variance*np.mean(b_list)/N
        for k in range(K):
            beta_vector[k] = estimated_noise_variance*b_list[k]/denominator  + np.sqrt(N*estimated_noise_variance*b_list[k])*Z/tao_0
        
        numerator = np.mean(variance*b_list)
        denominator1 = np.mean((tao_hat - tao_0)**2) - np.mean(variance*b_list)/N
        beta_constant = numerator/denominator1 + np.sqrt(N*numerator)*Z/tao_0

        #beyasian decision
        decision4 = []
  
        for k in range(K):
            if denominator1 <= 0:
                theta = 1
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)
            else:
                theta = N/(N+max((variance[k]*b_list[k])/denominator1,0))
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(1/denominator1+N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)




        
        
        beta_constant = max(0,beta_constant)
        beta_vector = np.maximum(0,beta_vector)
        
        theta_list = N/(N+beta_vector)
        theta = N/(N+beta_constant)
      
        tao_shunken_hat = tao_hat*theta + (1-theta)*tao_0

        tao_shunken_hat1 = tao_hat*theta_list + (1-theta_list)*tao_0
        
       
        decision1 = np.argwhere(tao_hat>(Z*np.sqrt(variance*b_list))/np.sqrt(N))
        decision2 = np.argwhere(tao_shunken_hat>(Z*np.sqrt(variance*b_list)*theta)/np.sqrt(N))
        decision3 = np.argwhere(tao_shunken_hat1>(Z*np.sqrt(variance*b_list)*theta_list)/np.sqrt(N))

        cost[0,index1,time_seq] = np.sum(true_tao[decision1])/optimal_cost
        cost[1,index1,time_seq] = np.sum(true_tao[decision2])/optimal_cost
        cost[2,index1,time_seq] = np.sum(true_tao[decision3])/optimal_cost
        cost[3,index1,time_seq] = np.sum(true_tao[decision4])/optimal_cost

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel(r'The value $\tau_0$',fontsize=15)
plt.ylabel('Optimality Ratio (OR)',fontsize=15)
x = [1,2,3,4,5]


y3 = np.mean(cost[2],axis=1)
plt.plot(x, y3, color = '#713948',marker = "^",label = "DTR-P")


y3 = np.mean(cost[1],axis=1)
plt.plot(x, y3, color = '#495373',linestyle = '-.',marker = "s",label = "DTR")

y3 = np.mean(cost[3],axis=1)
plt.plot(x, y3, color = '#E3738B',linestyle = '--',marker = "s",label = "Bayesian")



y3 = np.mean(cost[0],axis=1)
plt.plot(x, y3, color = '#8CA5EA',linestyle = '--', marker='o',label = "IHT")

plt.xticks([1,2,3,4,5],fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.savefig('performance_compare_with_tau0_uniform_feature.png',dpi=300,bbox_inches = 'tight')
plt.show()

### 1.2 Change with prior mean - Normal-Normal setting 

In [ ]:
K = 100 # number of experiments
N = 10 # number of samples in each experiment
std = 3 # standard deviation of noise
f_x = 4 # number of historical features
Z = t_dist.ppf(0.975,N - f_x - 2)   # critical value for two-tailed test at 95% confidence level

cost = np.zeros((5,5,1000))

for index1,anchor_mean in enumerate([1,2,3,4,5]):
    for time_seq in tqdm(range(1000)):

        #gerenate ture parameters
        true_tao = np.random.normal(anchor_mean,3, K)
        optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)]) 
        ture_theta_vector = np.random.uniform(-0.3,0.5,(K,f_x + 1))

        # Generate historical features and response variable
        X = np.ones((K,f_x+2,N))
        for i in range(K):
            for j in range(N):
                indicator = np.random.choice([0,1],1)
                if indicator == 0:
                    X[i,-1,j] = 0
                else:
                    X[i,-1,j] = 1
            hist_feature = np.random.uniform(0,1,(f_x,N))
            X[i,1:-1,:] = hist_feature
        Y = np.zeros((K,N))
        for i in range(K):
            for j in range(N):
                Y[i,j] = np.sum(ture_theta_vector[i] * X[i,0:-1,j]) + true_tao[i]*X[i,-1,j] + np.random.normal(0,std)

        #ols
        tao_hat = np.zeros(K)
        variance = np.zeros(K)
        p_value_list = np.zeros(K)
        b_list = np.zeros(K)

        variance_1 = np.zeros(K)

        for k in range(K):
            model = sm.OLS(Y[k], X[k].T).fit()
            tao_hat[k] = model.params[-1]
            p_value_list[k] = model.pvalues[-1]
            b_list[k] = N*np.linalg.inv(X[k] @ X[k].T + (10**-5) * np.eye(X[k].shape[0]))[-1,-1]

            variance[k] = np.sum(model.resid**2)/(N - f_x - 2)
            
        
        #personalised parameter and consistant parameter
        tao_0 = np.mean(tao_hat)
        beta_vector = np.zeros(K)

        estimated_noise_variance = np.mean(variance)
        denominator = np.mean((tao_hat - tao_0)**2) - estimated_noise_variance*np.mean(b_list)/N
        for k in range(K):
            beta_vector[k] = estimated_noise_variance*b_list[k]/denominator  + np.sqrt(N*estimated_noise_variance*b_list[k])*Z/tao_0
        
        numerator = np.mean(variance*b_list)
        denominator1 = np.mean((tao_hat - tao_0)**2) - np.mean(variance*b_list)/N
        beta_constant = numerator/denominator1 + np.sqrt(N*numerator)*Z/tao_0

        #bayesian decision
        decision4 = []
  
        for k in range(K):
            if denominator1 <= 0:
                theta = 1
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)
            else:
                theta = N/(N+max((variance[k]*b_list[k])/denominator1,0))
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(1/denominator1+N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)





        beta_constant = max(0,beta_constant)
        beta_vector = np.maximum(0,beta_vector)
        
        theta_list = N/(N+beta_vector)
        theta = N/(N+beta_constant)
       
        tao_shunken_hat = tao_hat*theta + (1-theta)*tao_0

        tao_shunken_hat1 = tao_hat*theta_list + (1-theta_list)*tao_0
    
        
        decision1 = np.argwhere(tao_hat>(Z*np.sqrt(variance*b_list))/np.sqrt(N))
        decision2 = np.argwhere(tao_shunken_hat>(Z*np.sqrt(variance*b_list)*theta)/np.sqrt(N))
        decision3 = np.argwhere(tao_shunken_hat1>(Z*np.sqrt(variance*b_list)*theta_list)/np.sqrt(N))

        cost[0,index1,time_seq] = np.sum(true_tao[decision1])/optimal_cost
        cost[1,index1,time_seq] = np.sum(true_tao[decision2])/optimal_cost
        cost[2,index1,time_seq] = np.sum(true_tao[decision3])/optimal_cost
        cost[3,index1,time_seq] = np.sum(true_tao[decision4])/optimal_cost

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel(r'The value $\tau_0$',fontsize=15)
plt.ylabel('Optimality Ratio (OR)',fontsize=15)
x = [1,2,3,4,5]


y3 = np.mean(cost[2],axis=1)
plt.plot(x, y3, color = '#713948',marker = "^",label = "DTR-P")


y3 = np.mean(cost[1],axis=1)
plt.plot(x, y3, color = '#495373',linestyle = '-.',marker = "s",label = "DTR")

y3 = np.mean(cost[3],axis=1)
plt.plot(x, y3, color = '#E3738B',linestyle = '--',marker = "s",label = "Bayesian")



y3 = np.mean(cost[0],axis=1)
plt.plot(x, y3, color = '#8CA5EA',linestyle = '--', marker='o',label = "IHT")

plt.xticks([1,2,3,4,5],fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.savefig('performance_compare_with_tau0_feature.png',dpi=300,bbox_inches = 'tight')
plt.show()

### 1.3 Change with number of expriments

In [ ]:
cost = np.zeros((5,5,1000))

anchor_mean = 1 # mean of the true parameter
N = 10 # number of samples in each experiment
std = 3 # standard deviation of noise
f_x = 4 # number of historical features
Z = t_dist.ppf(0.975,N - f_x - 2) # critical value for two-tailed test at 95% confidence level
K_list = [20,40,60,80,100] # list of number of experiments

for index1,K in enumerate(K_list):
    for time_seq in tqdm(range(1000)):

        # Generate true parameters
        true_tao = np.random.normal(anchor_mean,3, K)
        optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)]) 
        ture_theta_vector = np.random.uniform(-0.3,0.5,(K,f_x + 1))

        # Generate historical features and response variable
        X = np.ones((K,f_x+2,N))
        for i in range(K):
            for j in range(N):
                indicator = np.random.choice([0,1],1)
                if indicator == 0:
                    X[i,-1,j] = 0
                else:
                    X[i,-1,j] = 1
            hist_feature = np.random.uniform(0,1,(f_x,N))
            X[i,1:-1,:] = hist_feature
        Y = np.zeros((K,N))
        for i in range(K):
            for j in range(N):
                Y[i,j] = np.sum(ture_theta_vector[i] * X[i,0:-1,j]) + true_tao[i]*X[i,-1,j] + np.random.normal(0,std)

        #ols
        tao_hat = np.zeros(K)
        variance = np.zeros(K)
        p_value_list = np.zeros(K)
        b_list = np.zeros(K)

        variance_1 = np.zeros(K)

        for k in range(K):
            model = sm.OLS(Y[k], X[k].T).fit()
            tao_hat[k] = model.params[-1]
            p_value_list[k] = model.pvalues[-1]
            b_list[k] = N*np.linalg.inv(X[k] @ X[k].T + (10**-5) * np.eye(X[k].shape[0]))[-1,-1]

            variance[k] = np.sum(model.resid**2)/(N - f_x - 2)


        #personalised parameter and consistant parameter
        tao_0 = np.mean(tao_hat)
        beta_vector = np.zeros(K)

        estimated_noise_variance = np.mean(variance)
        denominator = np.mean((tao_hat - tao_0)**2) - estimated_noise_variance*np.mean(b_list)/N
        for k in range(K):
            beta_vector[k] = estimated_noise_variance*b_list[k]/denominator  + np.sqrt(N*estimated_noise_variance*b_list[k])*Z/tao_0
        
        numerator = np.mean(variance*b_list)
        denominator1 = np.mean((tao_hat - tao_0)**2) - np.mean(variance*b_list)/N
        beta_constant = numerator/denominator1 + np.sqrt(N*numerator)*Z/tao_0

        #bayesian decision
        decision4 = []
  
        for k in range(K):
            if denominator1 <= 0:
                theta = 1
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)
            else:
                theta = N/(N+max((variance[k]*b_list[k])/denominator1,0))
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(1/denominator1+N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)




        
       
        beta_constant = max(0,beta_constant)
        beta_vector = np.maximum(0,beta_vector)
      
        theta_list = N/(N+beta_vector)
        theta = N/(N+beta_constant)
     
        tao_shunken_hat = tao_hat*theta + (1-theta)*tao_0

        tao_shunken_hat1 = tao_hat*theta_list + (1-theta_list)*tao_0
    
        
        decision1 = np.argwhere(tao_hat>(Z*np.sqrt(variance*b_list))/np.sqrt(N))
        decision2 = np.argwhere(tao_shunken_hat>(Z*np.sqrt(variance*b_list)*theta)/np.sqrt(N))
        decision3 = np.argwhere(tao_shunken_hat1>(Z*np.sqrt(variance*b_list)*theta_list)/np.sqrt(N))

        cost[0,index1,time_seq] = np.sum(true_tao[decision1])/optimal_cost
        cost[1,index1,time_seq] = np.sum(true_tao[decision2])/optimal_cost
        cost[2,index1,time_seq] = np.sum(true_tao[decision3])/optimal_cost
        cost[3,index1,time_seq] = np.sum(true_tao[decision4])/optimal_cost

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel(r'The number of experiments $K$',fontsize=15)
plt.ylabel('Optimality Ratio (OR)',fontsize=15)

x = [20,40,60,80,100]
y3 = np.mean(cost[2],axis=1)
plt.plot(x, y3, color = '#713948',marker = "^",label = "DTR-P")


y3 = np.mean(cost[1],axis=1)
plt.plot(x, y3, color = '#495373',linestyle = '-.',marker = "s",label = "DTR")

y3 = np.mean(cost[3],axis=1)
plt.plot(x, y3, color = '#E3738B',linestyle = '--',marker = "s",label = "Bayesian")



y3 = np.mean(cost[0],axis=1)
plt.plot(x, y3, color = '#8CA5EA',linestyle = '--', marker='o',label = "IHT")

plt.xticks(x,fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.savefig('performance_compare_with_K_feature.png',dpi=300,bbox_inches = 'tight')
plt.show()

### 1.4 Change with sample size

In [ ]:
K = 100 # number of experiments
N_list = [10,15,20,25,30] # list of number of samples in each experiment
cost = np.zeros((5,5,1000))

anchor_mean = 1 # mean of the true parameter
std = 3 # standard deviation of noise
f_x = 4 # number of historical features
Z = t_dist.ppf(0.975,N - f_x - 2) # critical value for two-tailed test at 95% confidence level

for index1,N in enumerate(N_list):
    for time_seq in tqdm(range(1000)):

        # Generate true parameters
        true_tao = np.random.normal(anchor_mean,3, K)
        optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)]) 
        ture_theta_vector = np.random.uniform(-0.3,0.5,(K,f_x + 1))

        # Generate historical features and response variable
        X = np.ones((K,f_x+2,N))
        for i in range(K):
            for j in range(N):
                indicator = np.random.choice([0,1],1)
                if indicator == 0:
                    X[i,-1,j] = 0
                else:
                    X[i,-1,j] = 1
            hist_feature = np.random.uniform(0,1,(f_x,N))
            X[i,1:-1,:] = hist_feature
        Y = np.zeros((K,N))
        for i in range(K):
            for j in range(N):
                Y[i,j] = np.sum(ture_theta_vector[i] * X[i,0:-1,j]) + true_tao[i]*X[i,-1,j] + np.random.normal(0,std)

        #ols
        tao_hat = np.zeros(K)
        variance = np.zeros(K)
        p_value_list = np.zeros(K)
        b_list = np.zeros(K)

        variance_1 = np.zeros(K)

        for k in range(K):
            model = sm.OLS(Y[k], X[k].T).fit()
            tao_hat[k] = model.params[-1]
            p_value_list[k] = model.pvalues[-1]
            b_list[k] = N*np.linalg.inv(X[k] @ X[k].T + (10**-5) * np.eye(X[k].shape[0]))[-1,-1]

            variance[k] = np.sum(model.resid**2)/(N - f_x - 2)


        # personalised parameter and consistant parameter
        tao_0 = np.mean(tao_hat)
        beta_vector = np.zeros(K)

        estimated_noise_variance = np.mean(variance)
        denominator = np.mean((tao_hat - tao_0)**2) - estimated_noise_variance*np.mean(b_list)/N
        for k in range(K):
            beta_vector[k] = estimated_noise_variance*b_list[k]/denominator  + np.sqrt(N*estimated_noise_variance*b_list[k])*Z/tao_0
        
        numerator = np.mean(variance*b_list)
        denominator1 = np.mean((tao_hat - tao_0)**2) - np.mean(variance*b_list)/N
        beta_constant = numerator/denominator1 + np.sqrt(N*numerator)*Z/tao_0

        # bayesian decision
        decision4 = []
  
        for k in range(K):
            if denominator1 <= 0:
                theta = 1
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)
            else:
                theta = N/(N+max((variance[k]*b_list[k])/denominator1,0))
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = 1/(1/denominator1+N/(variance[k]*b_list[k]))
                dist = norm(loc=posteri_mean, scale=np.sqrt(posteri_var))
                prob = dist.sf(0)  # survival function: P(X > x)
                if prob > 1 - 0.025:
                    decision4.append(k)




       
        beta_constant = max(0,beta_constant)
        beta_vector = np.maximum(0,beta_vector)

        theta_list = N/(N+beta_vector)
        theta = N/(N+beta_constant)
    
        tao_shunken_hat = tao_hat*theta + (1-theta)*tao_0

        tao_shunken_hat1 = tao_hat*theta_list + (1-theta_list)*tao_0
    

        decision1 = np.argwhere(tao_hat>(Z*np.sqrt(variance*b_list))/np.sqrt(N))
        decision2 = np.argwhere(tao_shunken_hat>(Z*np.sqrt(variance*b_list)*theta)/np.sqrt(N))
        decision3 = np.argwhere(tao_shunken_hat1>(Z*np.sqrt(variance*b_list)*theta_list)/np.sqrt(N))

        cost[0,index1,time_seq] = np.sum(true_tao[decision1])/optimal_cost
        cost[1,index1,time_seq] = np.sum(true_tao[decision2])/optimal_cost
        cost[2,index1,time_seq] = np.sum(true_tao[decision3])/optimal_cost
        cost[3,index1,time_seq] = np.sum(true_tao[decision4])/optimal_cost

In [ ]:
plt.figure(figsize=(8,6))
plt.xlabel(r'Sample size $N$',fontsize=15)
plt.ylabel('Optimality Ratio (OR)',fontsize=15)

x = [10,15,20,25,30]
y3 = np.mean(cost[2],axis=1)
plt.plot(x, y3, color = '#713948',marker = "^",label = "DTR-P")


y3 = np.mean(cost[1],axis=1)
plt.plot(x, y3, color = '#495373',linestyle = '-.',marker = "s",label = "DTR")

y3 = np.mean(cost[3],axis=1)
plt.plot(x, y3, color = '#E3738B',linestyle = '--',marker = "s",label = "Bayesian")



y3 = np.mean(cost[0],axis=1)
plt.plot(x, y3, color = '#8CA5EA',linestyle = '--', marker='o',label = "IHT")

plt.xticks(x,fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.savefig('performance_compare_with_N_feature.png',dpi=300,bbox_inches = 'tight')
plt.show()